<a href="https://colab.research.google.com/github/ngrisouard/.emacs.d/blob/master/Copy_of_pix2pix101.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The following is a demo of pretty much everything. 2 main steps: preprocessing and training/testing

# Image Preprocessing (Done Locally, ideally) 

To convert from Ponte's simulation .nc files to `combined/side-by-side` images (almost\*) ready for training.

\*Need to remove blank images by hand



## First part

Use `tools/SnapPrints.py`. This allows you to specify custom colorbar, shape of image and txt file for absolute paths of simulations you want to use. 

To execute, run in terminal from `p2p-data-manipulation` folder (details about arguments in `SnapPrints.py`, **Check if true**):

`python tools/SnapPrints.py --SimsToUse CustomSims.txt --ColourBar N3gauss --Shape Square`

Note: This part should be done locally, since the the .txt file contains absolute paths from Data2. 



## Second part
Use `tools/process.py` (for now, this needs to be run here using tensorflow, which is not on the workstation currently)

`python tools/process.py --input_dir ssh --b_dir wave --operation combine --output_dir combined_snaps`

Once tensorflow works locally, you can use `preprocess.sh` to do the above 2 in one step. This script also combines images , so we will run the script here. 

But before running anything here first we need to turn on the GPU and start a runtime:

  * *Runtime -> Change runtime type -> Hardware Accelerator -> GPU*
  * *Runtime -> Reset all runtimes*



In [0]:
# This cell defines different customized paths
# You will need to modify it 
# gd_pth = '/Users/NicoG/GDrive/'  # path to the Google Drive folder on machine
gd_pth = '/content/gdrive/'  # shoudl always be the same
# Below: path to Google Drive folder containing code and images
p2p_on_gd = os.path.join(gd_pth, 'GDrive/UofT/nico_pix2pix/')
# p2p_on_gd = 'My\ Drive/pix2pix/'  # Michael's original path

In [0]:
# upload folder to google drive first.
# You will need to change some paths if you change folder names
from google.colab import drive
import os

drive.mount(gd_pth)
preprocess = os.path.join(p2p_on_gd, 'tools/process.py')
ssh = os.path.join(p2p_on_gd, 'ssh')
wave = os.path.join(p2p_on_gd, 'wave')

#make folder for combined images
combined_snaps = os.path.join(p2p_on_gd, 'combined')
if not os.path.isdir(combined_snaps):
    os.mkdir(combined_snaps)

In [0]:
# This line below is to make sure the GPU is running. Usually the Nvidia Tesla
# T4 is running (better), sometime you get the Nvidia K80 which is still not bad
!nvidia-smi

# This part below check how much of the GPU is being used. If Util is 0% (almost
# always), you have the full GPUSometimes, Util = 95% (never happened to me
# though, but others online). That means you only have 5% of the GPU.

# You can ignore the details of this cell. Never changes, and is more of a check
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + 
       humanize.naturalsize( psutil.virtual_memory().available ), 
       " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(
     gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()

We also need to upload our folders to google drive and connect to it. If we keep files in the google colab 'cloud', it deletes after 12 hours, so we just use our drive as storage. This is pretty standard.

In [0]:
# Continuing to Second part
!python $preprocess --input_dir $ssh --b_dir $wave --operation combine --output_dir $combined_snaps
 # Above: $ is to use a variable as an argument, I think

After images are combined, we manually remove blank images (from starts of simulations) and create a train/test set from the combined_snaps folder generated above. 80/20 split is recommended. 

I have not automated this part since it's quite short. Put all images into a train folder, then cut every 5th image into a test folder.

# 2.  Training & Testing (Done on Scinet Graham/Google Colab, need GPU)

For this demo, we will use Google Colab, but it should give you a good idea of how the training/testing/error metric process works. Running code here first can be helpful to see how long a simulation will take  (progress printed in real time) which can give you an idea of how much time required for Scinet. From my experience, they are about the same speed. 

For Scinet, I made an .sh file (pix2pix.sh) that has worked before on the Graham cluster. You can modify it to do multiple runs or whatnot.



In [0]:
pix2pix = os.path.join(p2p_on_gd, '/pix2pix/pix2pix.py')

input_train = os.path.join(p2p_on_gd, 'train') # images to train on
output_train = os.path.join(p2p_on_gd, 'trained') # folder to save trained model
checkpoint = os.path.join(p2p_on_gd, 'trained')

input_test = os.path.join(p2p_on_gd, 'test')
output_test = os.path.join(p2p_on_gd, 'tested')

In [0]:
#if using rectangular images, add: --aspect_ratio=0.5
#save_freq optional. Default is 5000, where #steps = #images*#epochs. 
#I put it there cause everytime a model is saved, the overwritten model (~500MB) goes to trash and can fill up Google Drive storage fast.

!python $pix2pix --mode train --output_dir $output_train --max_epochs 500 --input_dir $input_train --which_direction AtoB --save_freq=20000 --aspect_ratio=0.5

In [0]:
#if using rectanglular images, add: --aspect_ratio=0.5
!python $pix2pix --mode test --output_dir $output_test --input_dir $input_test --aspect_ratio=0.5 --checkpoint $checkpoint

In [0]:
# Finally we use L1 (pixel difference averaged along each RGB channel) as our error metric
loss = os.path.join(p2p_on_gd, 'tools/loss.py')
image_folder = os.path.join(p2p_on_gd, 'tested')
!python $loss --image_dir $image_folder 

In [0]:
# load tensorboard for visualizing training (not testing)

!pip install -q tf-nightly-2.0-preview
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [0]:
# if using tensorboard locally, you do not need the above cell, or the below '%' or '$'
# we only care about generator_loss_L1
%tensorboard --logdir $output_train

To run multiple tensorboards at once: https://stackoverflow.com/questions/36182380/how-do-display-different-runs-in-tensorboard